<a href="https://colab.research.google.com/github/SumathiGit/Python-World/blob/main/RBM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [34]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data
import torch.nn.parallel
import numpy as np
import pandas as pd
from torch.autograd import Variable

In [35]:
movies = pd.read_csv("data/movies.dat", sep = '::', header = None, engine = 'python', encoding = 'latin-1')
users = pd.read_csv("data/users.dat", sep = '::', header = None, engine = 'python', encoding = 'latin-1')

In [36]:
ratings = pd.read_csv("data/ratings.dat", sep = "::::::", header = None, engine = 'python')

In [37]:
training_set = pd.read_csv("data/u1.base", delimiter='\t')
training_set = np.array(training_set, dtype='int')

In [38]:
testing_set = pd.read_csv("data/u1.test", delimiter='\t')
testing_set = np.array(testing_set, dtype='int')

In [39]:
#Getting the number of users and movies
total_no_of_user = int(max(max(training_set[:,0]),max(testing_set[:,0])))
total_no_of_movies = int(max(max(training_set[:,1]),max(testing_set[:,1])))

In [40]:
#Converting the data ito matrix  with user and movies column
def convert(data):
    new_data = []
    for user_id in range(1, total_no_of_user+1):
        movie_ids = data[:,1][data[:,0] == user_id]
        rating_ids = data[:,2][data[:,0] == user_id]
        ratings = np.zeros(total_no_of_movies)
        ratings[movie_ids -1] = rating_ids
        new_data.append(list(ratings))
    return new_data

In [41]:
#So we get our training_set and testing_set in a list of list format (2dim)  
training_set = convert(training_set)
testing_set = convert(testing_set)

In [42]:
#now Convert them into torch Tensors
training_set = torch.FloatTensor(training_set)
testing_set = torch.FloatTensor(testing_set)

print(testing_set.type)

<built-in method type of Tensor object at 0x7fc27d3b4050>


In [43]:
#Converting the rating into binary rating 1>like 0>dislike
training_set[training_set == 0] = -1
training_set[training_set == 1] = 0
training_set[training_set == 2] = 0
training_set[training_set >= 3] = 1

testing_set[testing_set == 0] = -1
testing_set[testing_set == 1] = 0
testing_set[testing_set == 2] = 0
testing_set[testing_set >= 3] = 1
print(training_set.type)


<built-in method type of Tensor object at 0x7fc273f5b410>


In [71]:
#Bernoulli example
a = torch.empty(3, 3).uniform_(0, 1)  # generate a uniform random matrix with range [0, 1]
a

tensor([[0.0086, 0.5803, 0.4745],
        [0.9648, 0.7528, 0.7471],
        [0.4922, 0.0633, 0.3109]])

In [72]:
print(a.type)

<built-in method type of Tensor object at 0x7fc273f28230>


In [77]:
torch.bernoulli(a)       #Draws a binary random number (0 or 1) for given random matrix.

tensor([[0., 1., 0.],
        [1., 1., 1.],
        [0., 0., 1.]])

In [59]:
class RBM():
    def __init__(self, input_nodes, hidden_nodes):      #Initialize by giving random weights for the input node >> training_set , hiddennode =100
        self.weight = torch.randn(hidden_nodes, input_nodes)
        self.hidden_bias = torch.randn(1, hidden_nodes)
        self.input_bias = torch.randn(1, input_nodes)
    
    def sample_hidden(self, x):      #Visible nodes(Input layer)
        wx = torch.mm(x, self.weight.t()) #First step is to multiply our randon weight with the input>x and then do a transpose for getting matrix form of output
        activation = wx + self.hidden_bias.expand_as(wx) #input > Input * weight + bias
        prob_hidden_given_visible = torch.sigmoid(activation)
        return prob_hidden_given_visible, torch.bernoulli(prob_hidden_given_visible) #This code returns the bernoulli martx as well as sample
    
    def sample_input(self, y):       #visible nodes(Input layer)
        wy = torch.mm(y, self.weight)
        activation = wy + self.input_bias.expand_as(wy)
        prob_visible_given_hidden = torch.sigmoid(activation)
        return prob_visible_given_hidden, torch.bernoulli(prob_visible_given_hidden)
    
    def train(self, visible0, visiblek, probhidden0, prodhiddenk): #Passing the parameters to trsin th model
        self.weight += (torch.mm(visible0.t(), probhidden0) - torch.mm(visiblek.t(), prodhiddenk)).t()
        self.input_bias += torch.sum((visible0 - visiblek), 0)
        self.hidden_bias += torch.sum((probhidden0 - prodhiddenk) , 0)
        

In [60]:
input_nodes = len(training_set[0])
hidden_nodes = 100        #corelate the input and extract 100 features 
batch_size = 100
rbm = RBM(input_nodes, hidden_nodes)

In [78]:
epochs = 10
for epoch in range(1, epochs +1):
    train_loss = 0
    s = 0. #Float will be used to normalize the loss
    for user_id in range(0, total_no_of_user - batch_size , batch_size):
        vk = training_set[user_id: user_id + batch_size]
        v0 = training_set[user_id: user_id + batch_size]
        ph0, _ = rbm.sample_hidden(v0) #visible node
        for k in range(10):  #K steps of contrastive divergence >>  we have passed the input to a neuron or a feature.>> Then it gets an output then input and output is compared.
            _,hk = rbm.sample_hidden(vk)
            _,vk = rbm.sample_input(hk)
            vk[v0 < 0] = v0[v0 < 0]
        phk, _ = rbm.sample_hidden(vk) 
        rbm.train(v0, vk, ph0, phk)
        train_loss += torch.mean(torch.abs(v0[v0>=0] - vk[v0>=0]))
        s += 1.
    print("epoch: " +str(epoch)+" loss: " +str(train_loss/s))




      #visible0 >> input vector containing the ratings of all the movies by one user 
      #visiblek >>visible node obtained after k iteration
      #probhidden0 >> Vector probabilities that the first iteration >the hidden nodes equal one give the values
      #probhiddenk >>probabilities of hidden node after k sample give the values of visible nodes



epoch: 1 loss: tensor(0.2466)
epoch: 2 loss: tensor(0.2473)
epoch: 3 loss: tensor(0.2469)
epoch: 4 loss: tensor(0.2450)
epoch: 5 loss: tensor(0.2458)
epoch: 6 loss: tensor(0.2467)
epoch: 7 loss: tensor(0.2469)
epoch: 8 loss: tensor(0.2445)
epoch: 9 loss: tensor(0.2464)
epoch: 10 loss: tensor(0.2463)


In [82]:
#Testing the RBM

test_loss = 0
s=0. 
for id_user in range(total_no_of_user):    #Just passing the testing_Set
  v = training_set[id_user:id_user+1]
  vt = testing_set[id_user:id_user+1]
  if len(vt[vt>=0]) > 0:
    _,h =rbm.sample_hidden(v)
    _,v = rbm.sample_input(h)
    test_loss += torch.mean(torch.abs(vt[vt>=0] - v[vt>=0]))
    s +=1.
print("test loss:" +str(test_loss/s))

test loss:tensor(0.2304)


In [ ]:
"""
dataset source https://grouplens.org/datasets/movielens/
dataset contains movies, users, ratings for more than 20k movies with 1000 of users
We are buliding the boltzmann machine to understand the co-relaton between the inputs and extract the features out of it
Here in this code we are initializing with the training and testing datasets and give random weights and weights to the input and hidden neurons and train them to minimize the loss
 
"""
